In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')

In [3]:
train.head()

,PID,Date,Event
0,1028890,201101,2186
1,1028890,201101,7087
2,1028890,201101,4848
3,1028890,201101,2214
4,1028890,201102,7087


In [4]:
train_df = pd.get_dummies(train['Event'])

In [5]:
train_df.shape

(766787, 6472)

In [6]:
trainX = pd.DataFrame()

In [7]:
#train_df['PID'] = train['PID'] 
trainX['PID'] = train['PID']

In [8]:
#train_df['Year'] = (train['Date']/100).astype(int)
trainX['Year'] = (train['Date']/100).astype(int)

In [9]:
#train_df['Year'].head(2)
trainX['Year'].head(2)

0    2011
1    2011
Name: Year, dtype: int64

In [10]:
#train_df['Month'] = (train['Date']%100).astype(int)
trainX['Month'] = (train['Date']%100).astype(int)

In [11]:
#train_df['Month'].head()

In [12]:
train_df.shape

(766787, 6472)

In [13]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 766787 entries, 0 to 766786
Columns: 6472 entries, 0001 to V910
dtypes: uint8(6472)
memory usage: 4.6 GB


In [14]:
#train_df.to_csv('Full_data.csv',index=False)

In [15]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), [0,2]]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 2])
    return np.array(dataX), np.array(dataY)

In [16]:
#trainX = train_df[['PID','Year','Month']]

In [17]:
#train_df.drop(['PID','Year','Month'],axis=1,inplace=True)

In [18]:
trainX.shape

(766787, 3)

In [19]:
trainX.head()

,PID,Year,Month
0,1028890,2011,1
1,1028890,2011,1
2,1028890,2011,1
3,1028890,2011,1
4,1028890,2011,2


In [20]:
trainX1=pd.get_dummies(trainX['PID'])

In [21]:
trainX1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 766787 entries, 0 to 766786
Columns: 3000 entries, 1000001 to 1031900
dtypes: uint8(3000)
memory usage: 2.1 GB


In [22]:
trainX1['Year'] = (train['Date']/100).astype(int)
trainX1['Month'] = (train['Date']%100).astype(int)

In [23]:
trainX1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 766787 entries, 0 to 766786
Columns: 3002 entries, 1000001 to Month
dtypes: int64(2), uint8(3000)
memory usage: 2.2 GB


In [24]:
trainX1.shape

(766787, 3002)

In [25]:
trainX1 = trainX.values

In [26]:
trainX1.shape

(766787, 3)

In [27]:
trainX1 = np.reshape(trainX1,(trainX1.shape[0],3,1))

In [28]:
trainX1.shape

(766787, 3, 1)

In [29]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow as tf
from keras.layers import Dense,LSTM
from keras.models import Sequential

Using TensorFlow backend.


In [30]:
reg = Sequential()

In [31]:
reg.add(LSTM(units =400,activation = 'sigmoid',input_shape = (None , 1)))

In [32]:
reg.add(Dense(units = 6472,kernel_initializer = 'uniform'))
reg.add(Dense(units = 6472 , kernel_initializer = 'uniform'))
reg.add(Dense(units=6472,kernel_initializer='uniform', activation='softmax'))
reg.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
reg.fit(trainX1,train_df.values,batch_size = 1000,epochs = 10)
#test = pd.read_csv('test.csv')

Epoch 1/10
186000/766787 [======>.......................] - ETA: 362s - loss: 15.4606 - categorical_accuracy: 0.0385

In [ ]:
reg.save('LSTM_V4.h5')

In [ ]:
test_df = pd.DataFrame()

In [ ]:
test_df['PID']=train['PID'].unique()

In [ ]:
test_df.info()

In [ ]:
test_df['Year']=2014

In [ ]:
test_df['Month']= 01

In [ ]:
test_df.head()

In [ ]:
test_df.shape

In [ ]:
test_df2=test_df.values

In [ ]:
test_df21 = np.reshape(test_df2,(test_df2.shape[0],1,1))

In [ ]:
test_df21.shape

In [ ]:
y_pred1 = reg.predict(test_df21,batch_size=1000)

In [ ]:
y_pred1.shape

In [ ]:
yy_pred1=y_pred1


In [ ]:
pred = pd.DataFrame(data=yy_pred1)
pred['PID'] = test_df['PID']
submit_pred = pred.loc[:,pred.columns != 'PID'].apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:1].index, index=['Event'+str(x) for x in range(1,2)]),axis=1).reset_index()
submit_pred.drop('index',inplace=True, axis=1)


In [ ]:
# setting column names in sequence
submit_pred['PID'] = test_df['PID']
cols = submit_pred.columns.tolist()
cols = cols[-1:] + cols[:-1]
submit_pred = submit_pred[cols]

In [ ]:
submit_pred.head()

In [ ]:
test_df['Month']= 02
print(test_df.head())
print(test_df.shape)
test_df3=test_df.values
test_df31 = np.reshape(test_df3,(test_df3.shape[0],test_df3.shape[1],1))
print(test_df31.shape)
y_pred2 = reg.predict(test_df31,batch_size=1000)
print(y_pred2.shape)
yy_pred2=y_pred2
pred2 = pd.DataFrame(data=yy_pred2)
pred2['PID'] = test_df['PID']
submit_pred2 = pred2.loc[:,pred2.columns != 'PID'].apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:1].index, index=['Event'+str(x) for x in range(2,3)]),axis=1).reset_index()
submit_pred2.drop('index',inplace=True, axis=1)


In [ ]:
submit_pred2.head()

In [ ]:
test_df['Month']= 03
print(test_df.head())
print(test_df.shape)
test_df4=test_df.values
test_df41 = np.reshape(test_df4,(test_df4.shape[0],test_df4.shape[1],1))
print(test_df41.shape)
y_pred3 = reg.predict(test_df41,batch_size=1000)
print(y_pred3.shape)
yy_pred3=y_pred3
pred3 = pd.DataFrame(data=yy_pred3)
pred3['PID'] = test_df['PID']
submit_pred3 = pred3.loc[:,pred3.columns != 'PID'].apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:1].index, index=['Event'+str(x) for x in range(3,4)]),axis=1).reset_index()
submit_pred3.drop('index',inplace=True, axis=1)
submit_pred3.head()
